## Results Data Scraping Notebook

This notebook uses links in the event_links.csv file to scrape the results table on each page, wrangle those results, and save them as a csv.

### Library Imports

In [25]:
using CSV
using DataFrames
using HTTP, Gumbo, Cascadia
using DataFrames
using DataStructures: OrderedDict

### Scraping Results

In [26]:
# Read in events.csv
event_links = CSV.read("event_links.csv", DataFrame)
event_links

,Column1,Event_Id,Link
,Int64,Int64,String
1,1,1,https://results.runningevents.co.nz/list/tauranga/2022/21km
2,2,1,https://results.runningevents.co.nz/list/tauranga/2022/10km
3,3,2,https://results.runningevents.co.nz/list/devonport/2022/21km
4,4,2,https://results.runningevents.co.nz/list/devonport/2022/10km
5,5,3,https://results.runningevents.co.nz/list/runorewa/2022/15km
6,6,3,https://results.runningevents.co.nz/list/runorewa/2022/10km
7,7,4,https://results.runningevents.co.nz/list/ccwellington/2020/5km
8,8,5,https://results.runningevents.co.nz/list/ccchristchurch/2021/5km
9,9,6,https://results.runningevents.co.nz/list/ccauckland/2020/5km


In [27]:
# Function that converts the HTML table on a page to a dataframe
# Code used from here: https://gist.github.com/scls19fr/9ea2fd021d5dd9a97271da317bff6533
function extract_table(table_results)

    tables = DataFrame[]

    for helm_table in table_results
        column_names = String[]
        d_table = OrderedDict{String, Vector{String}}()
        for (i, row) in enumerate(eachmatch(Selector("tr"), helm_table))
            if (i == 1)
                    for (j, colh) in enumerate(eachmatch(Selector("th"), row))
                        colh_text = strip(nodeText(colh))
                        while (colh_text in column_names)  # Column header must be unique
                            colh_text = colh_text * "_2"
                        end
                        push!(column_names, colh_text)
                end
            else
                if (i == 2)
                    for colname in column_names
                        d_table[colname] = Vector{String}()
                    end
                end
                for (j, col) in enumerate(eachmatch(Selector("td"), row))
                    col_text = strip(nodeText(col))
                    colname = column_names[j]
                    push!(d_table[colname], col_text)
                end
            end
        end
        df = DataFrame(d_table)
        push!(tables, df)
    end
    
    return tables[1]

end

extract_table (generic function with 1 method)

In [28]:
# Initialise array to hold dataframes
results_dfs = Array{DataFrame}(undef, size(event_links, 1))

# Iterate over each link 
for i in 1:size(event_links, 1)
    
    # Fetch the page
    results_page = HTTP.get(event_links[i, 3])
    
    # Obtain the body section of the response
    parsed_page = results_page.body |>
        String |>
        parsehtml
    
    # Extract the table node from the XML document
    table_results = eachmatch(Selector("table"), parsed_page.root)
    
    # Extract the table 
    table_df = extract_table(table_results)
    
    # Add the Event_Id to all rows in the dataframe
    table_df[:, "Event_Id"] .= event_links[i, 2]
    
    # Add the table to the results array
    results_dfs[i] = table_df
       
end

### Wrangling the Data

In [29]:
result_df = results_dfs[1] 

# Iteratively build the results dataframe
for i in 2:size(results_dfs, 1)
   result_df = vcat(result_df, results_dfs[i])
end

In [30]:
result_df

,#,Bib,Name,Time,Place,Gender,G/Pl,Division
,String,String,String,String,String,String,String,String
1,,1623,Ben Twyman,01:14:22,1,Male,1,M3039
2,,1588,Ewan Glassey,01:15:43,2,Male,2,M3039
3,,7854,Joel Natzke,01:17:03,3,Male,3,M2029
4,,1496,Brad Dixon,01:17:12,4,Male,4,M4049
5,,7880,Eva Goodisson,01:17:25,5,Female,1,F2029
6,,1399,Duncan Baxter,01:19:30,6,Male,5,M3039
7,,1408,Quintin Moolman,01:20:14,7,Male,6,M3039
8,,1630,Luke Williams,01:20:20,8,Male,7,M5059
9,,1442,Adam Hazlett,01:20:38,9,Male,8,M4049


In [31]:
# Remove the '#' column as the information it is meant to contain is the same as the 'Place' column
# Also remove the last column as it contains a link that will not be needed

result_df = select!(result_df, Not([:"#", :""]))

first(result_df, 5)

,Bib,Name,Time,Place,Gender,G/Pl,Division,D/Pl,Event_Id
,String,String,String,String,String,String,String,String,Int64
1,1623,Ben Twyman,01:14:22,1,Male,1,M3039,1,1
2,1588,Ewan Glassey,01:15:43,2,Male,2,M3039,2,1
3,7854,Joel Natzke,01:17:03,3,Male,3,M2029,1,1
4,1496,Brad Dixon,01:17:12,4,Male,4,M4049,1,1
5,7880,Eva Goodisson,01:17:25,5,Female,1,F2029,1,1


In [32]:
# Write dataframe to a .csv file
CSV.write("results.csv", result_df)

"results.csv"